# Tree traversal

Tree traversal refers to visiting every node in the tree and processing its data. Depending on the goal of the traversal, "processing" can mean almost anything - the most basic example is just printing the value of the node. 

**Running time**: `O(n)` for `n` nodes in the tree; we must visit every node in the tree.

Traversing the tree creates an ordering of its nodes; there are several possible orderings.
- **Pre-order**: Process the current node, visit the left child, visit the right child.
- **In-order**: Visit the left child, process the current node, visit the right child.
- **Post-order**: Visit the left child, visit the right child, process the current node.
- **Level-order**: All nodes at the given level are visited before any of their children.

> The trace of a traversal is called a sequentialisation of the tree. The traversal trace is a list of each visited root. No one sequentialisation according to pre-, in- or post-order describes the underlying tree uniquely. Given a tree with distinct elements, either pre-order or post-order paired with in-order is sufficient to describe the tree uniquely. ([Wikipedia](https://en.wikipedia.org/wiki/Tree_traversal#In-order_(LNR)))

The most straightforward way to implement pre/in/post-order traversals uses recursion. Note that below, we can easily change from pre- to in- or post- order by simply changing where we call to `print()`, and can reverse the order by swapping the left and right:

In [1]:
"""
     4
   /   \
  2     6
 / \   / \
1   3 5   7
"""
class TreeNode:
    def __init__(self, val=None, parent=None, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.parent = parent # Some BSTs use nodes without references to parents

        
tree_dict = {val: TreeNode(val) for val in range(1,8)}
tree_dict[4].left = tree_dict[2]
tree_dict[4].right = tree_dict[6]
tree_dict[2].left = tree_dict[1]
tree_dict[2].right = tree_dict[3]
tree_dict[6].left = tree_dict[5]
tree_dict[6].right = tree_dict[7]
tree_dict[1].parent = tree_dict[2]
tree_dict[2].parent = tree_dict[4]
tree_dict[3].parent = tree_dict[2]
tree_dict[5].parent = tree_dict[6]
tree_dict[6].parent = tree_dict[4]
tree_dict[7].parent = tree_dict[6]
root = tree_dict[4]

In [2]:
def pre_order_recursive(node: TreeNode):
    if not node: return
    print(node.val, end=" ")
    pre_order_recursive(node.left)
    pre_order_recursive(node.right) 
    
print(f"Pre-order", end=" ")
pre_order_recursive(root)
print(" ")

Pre-order 4 2 1 3 6 5 7  


In [3]:
def in_order_recursive(node: TreeNode):
    if not node: return
    in_order_recursive(node.left)
    print(node.val, end=" ")
    in_order_recursive(node.right)

print(f"In-order:", end=" ")
in_order_recursive(root)
print(" ")

In-order: 1 2 3 4 5 6 7  


In [4]:
def post_order_recursive(node: TreeNode):
    if not node: return
    post_order_recursive(node.left)
    post_order_recursive(node.right)
    print(node.val, end=" ")

print(f"Post-order:", end=" ")
post_order_recursive(root)
print(" ")

Post-order: 1 3 2 5 7 6 4  


Traversals can also be implemented explicitly using a stack. Iterative versions are not as straightforward and make for great interview questions:

In [5]:
def pre_order_iterative(node: TreeNode):
    traversal = []
    stack = [node]
    while stack:
        curr = stack.pop()
        if curr:
            traversal.append(curr.val)
            stack.append(curr.right)
            stack.append(curr.left)
    return traversal
print(f"Pre-order: {pre_order_iterative(root)}") 

Pre-order: [4, 2, 1, 3, 6, 5, 7]


In [6]:
def in_order_iterative(node: TreeNode):
    traversal = []
    stack = []
    while node or stack:
        if node:
            stack.append(node)
            node = node.left
        else:
            node = stack.pop()
            traversal.append(node.val)
            node = node.right
    return traversal        
print(f"In-order: {in_order_iterative(root)}")

In-order: [1, 2, 3, 4, 5, 6, 7]


In [7]:
def post_order_iterative(node: TreeNode):
    traversal = []
    stack = []
    prev_node = None
    while stack or node:
        if node:
            stack.append(node)
            node = node.left
        else:
            peek = stack[-1] 
            if peek.right and prev_node != peek.right:
                node = peek.right
            else:
                traversal.append(peek.val)
                prev_node = stack.pop()
    return traversal
 
print(f"Post-order: {post_order_iterative(root)}")

Post-order: [1, 3, 2, 5, 7, 6, 4]


Level order traversal is implemented iteratively, but using a queue (most easily via `collections.deque()` in Python), and can be used for breadth-first searches:

In [ ]:
from collections import deque

def level_order_iterative(node: TreeNode):
    q = deque([node])
    while q:
        curr = q.popleft()
        print(curr.val, end =" ")
        q.append(curr.left) if curr.left else None
        q.append(curr.right) if curr.right else None 
level_order_iterative(root)

Also, remember that each type of traversal can be implemented using an array instead of a node type if given, though this tends to be more wasteful if the BST isn't complete:

In [10]:
from typing import List

tree_as_arr = [4,2,6,1,3,5,7]

def pre_order_arr(tree: [int], i: int):
    if not (0 <= i <=len(tree)-1):
        return
    print(tree[i], end =" ")
    pre_order_arr(tree, 2*i+1)
    pre_order_arr(tree, 2*i+2)
pre_order_arr(tree_as_arr, 0)

4 2 1 3 6 5 7 

#### Exercise (CLRS 12.1-3)
Give a nonrecursive algorithm that performs an inorder tree walk. (Hint: An easy solution uses a stack as an auxiliary data structure.  A more complicated, but elegant, solution uses no stack but assumes that we can test two pointers for equality.)

</br>
<details>
<summary><b>Click to view summary.</b></summary>
The stack-based approach is shown above. For the two-pointer solution, CLRS makes a critical distinction in its definition of BSTs:
> We can represent such a tree by a linked data structure in which each node is an object.  In addition to a key and satellite data, each node contains attributes `left`, `right`, and `p` that point to the nodes corresponding to its left child, its right child, and its parent, respectively. (p. 286-287)

With a pointer to the parent, we can leverage the fact that the root of the tree is the only node without a parent, and use the following approach:
```
Start at the root node, and do the following until our current node is nil:
    - If there's a left and we just came from the parent, go left.
    - If there's no left or we just came from the left:
        - print this node's value.
        - If there is a right, go right
    - If we just came from the right or there is no right:
    - Otherwise Go to the parent
 ```
    
Implemented:
```python3
"""
      0
    /    \
   1      2
  / \    / \ 
 3   4  5   6 
"""

def inorder_traversal(root):
    current = root
    past = None
    while current:
        # Go left if we just came from the parent and a left child exists.
        if (past == current.parent) and current.left:
            past = current
            current = current.left
            continue
        # Print the current node's value if we just came from the left or can't go left.
        if (past == current.left) or (not current.left):
            print(current.val, end=" ")
        # Go right if we just came from the left and can go right.
        if (past == current.left) and current.right:
            past = current
            current = current.right
            continue
        # Otherwise return to the parent 
        past = current
        current = current.parent
inorder_traversal(root)    
```
</details>